In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import roc_auc_score
import dagshub
import mlflow
import mlflow.sklearn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [25]:
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')

In [26]:
dagshub.init(repo_owner='gioeba', repo_name='IEEE-CIS-Fraud-Detection', mlflow=True)

Initialized MLflow to track repo "gioeba/IEEE-CIS-Fraud-Detection"

Repository gioeba/IEEE-CIS-Fraud-Detection initialized!

# Cleaning

In [27]:
df = train_transaction.merge(train_identity, on='TransactionID', how='left')
y = df['isFraud']
X = df.drop(columns=['isFraud', 'TransactionID']).copy()

# Feature Engineering

In [28]:
logged_fe = 'runs:/2251e67e9c084b99b4e492e146e576a0/fe'
feature_engineering_model = mlflow.sklearn.load_model(logged_fe)

# Feature Selection

In [29]:
logged_preprocessor = 'runs:/de686b00b14a4c7aac22431740c0ebb5/pre-processor'
dynamic_preprocessor = mlflow.sklearn.load_model(logged_preprocessor)

In [30]:
clf_pipeline = Pipeline([
    ('feature_engineering', feature_engineering_model),
    ('preprocessing', dynamic_preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=10,
        max_depth=12,
        class_weight='balanced',
        n_jobs=-1,
        random_state=42
    ))
])

# Training

In [31]:
mlflow.set_experiment("ieee-random-forest")
with mlflow.start_run(run_name="rf-depth-12-n10-streamlined"):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    auc_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
        clf_pipeline.fit(X_train, y_train)
        y_prob = clf_pipeline.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_prob)
        auc_scores.append(auc)
        
        print(f"Fold {fold} AUC: {auc:.4f}")
        mlflow.log_metric(f"fold_{fold}_auc", auc)
    
    avg_auc = np.mean(auc_scores)
    mlflow.log_metric("avg_roc_auc", avg_auc)
    mlflow.log_param("model", "RandomForest")
    mlflow.log_param("max_depth", 12)
    mlflow.log_param("n_estimators", 10)
    print(f"\nAverage AUC across folds: {avg_auc:.4f}")

Fold 1 AUC: 0.8972
Fold 2 AUC: 0.8975
Fold 3 AUC: 0.8933
Fold 4 AUC: 0.8960
Fold 5 AUC: 0.8967

Average AUC across folds: 0.8961
🏃 View run rf-depth-12-n10-streamlined at: https://dagshub.com/gioeba/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/1/runs/df218c762fb547708e25009d1fb25fb3
🧪 View experiment at: https://dagshub.com/gioeba/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/1
